In [2]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [5]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import os
import shutil

airbus_ship_detection_path = kagglehub.competition_download('airbus-ship-detection')

print('Data source import complete.')


Data source import complete.


In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Adatok letöltése, előkészítése

In [7]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from skimage import io


random_seed = 42
np.random.seed(random_seed)

"""
dataset = pd.read_csv('../input/airbus-ship-detection/train_ship_segmentations_v2.csv')


train_split = 0.8
valid_split = 0.1
test_split = 0.1

#esetleg shuffle

#X_train = [;]
X_test =
X_valid =

Y_train =
Y_test =
Y_valid =
"""
class ShipDataset(Dataset):


    def __init__(self, dataframe, image_dir, transform=None):

        self.ships_frame = dataframe
        self.image_dir = image_dir

    def __len__(self):
        return len(self.ships_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.image_dir,
                                self.ships_frame.iloc[idx, 0])
        image = io.imread(img_name)
        ships = self.ships_frame.iloc[idx, 1:]
        ships = convert_to_2d_tensor(ships)
        sample = {'image': image, 'ships': ships}


        return sample

def convert_to_2d_tensor(data):
    # Step 1: Split the string into individual numbers and convert to integers
    numbers = list(map(int, data.split()))

    # Step 2: Reshape the list into pairs (start, length)
    arr = np.array(numbers).reshape(-1, 2)  # Each pair is (start position, length)

    # Step 3: Convert the NumPy array to a PyTorch tensor
    tensor = torch.tensor(arr, dtype=torch.int32)

    return tensor